In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
gdf = gpd.read_file("VA_Overlap.json")

In [3]:
gdf.head()

,CName1,CName2,ID,geometry
0,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"MULTIPOLYGON (((-78.46688 38.06782, -78.46687 ..."
1,va-albemarle-0022,va-augusta-0169,va-overlap-1,"MULTIPOLYGON (((-78.83475 38.05003, -78.83458 ..."
2,va-albemarle-0022,va-nelson-1504,va-overlap-2,"POLYGON ((-78.80658 37.99217, -78.80659 37.992..."
3,va-albemarle-0023,va-charlottesvillecity-0347,va-overlap-3,"MULTIPOLYGON (((-78.50561 38.01038, -78.50529 ..."
4,va-albemarle-0025,va-rockingham-2049,va-overlap-4,"MULTIPOLYGON (((-78.74943 38.20659, -78.74892 ..."


In [4]:
# OVERLAP “{lat};{lng};{precinct-cname-a};{precinct-cname-b}”
newgdf = gpd.GeoDataFrame(columns=["CName1", "CName2", "ID", "geometry"])
newgdf

,CName1,CName2,ID,geometry


In [5]:
for index, row in gdf.iterrows():
    g = row["geometry"]
    c1, c2 = row["CName1"], row["CName2"]
    ID = row["ID"]
    
    if g.geom_type == "MultiPolygon":
        for shape in g:
            newgdf = newgdf.append({"CName1":c1, "CName2":c2, "ID":ID, "geometry":shape}, ignore_index=True)
    else:
        newgdf = newgdf.append({"CName1":c1, "CName2":c2, "ID":ID, "geometry":g}, ignore_index=True)
newgdf.head()

,CName1,CName2,ID,geometry
0,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"POLYGON ((-78.46688 38.06782, -78.46687 38.067..."
1,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"POLYGON ((-78.46953 38.06793, -78.46980 38.067..."
2,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"POLYGON ((-78.47101 38.06974, -78.47175 38.070..."
3,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"POLYGON ((-78.47341 38.06974, -78.47345 38.069..."
4,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"POLYGON ((-78.47476 38.06828, -78.47475 38.068..."


In [7]:
newdf = pd.DataFrame(columns=["Info", "ID"])
for index, row in newgdf.iterrows():
    x, y  = row["geometry"].centroid.coords[0]
    c1, c2 = row["CName1"], row["CName2"]
    ID = row["ID"]
    info = str(x) + ";" + str(y) + ";" + c1 + ";" + c2
    
    newdf = newdf.append({"ID":ID, "Info":info}, ignore_index=True)

newdf["State"] = "va"
newdf["Type"] = "OVERLAP"
newdf.head()

,Info,ID,State,Type
0,-78.46738969917081;38.0677285654716;va-albemar...,va-overlap-0,va,OVERLAP
1,-78.47100397149606;38.067944815142134;va-albem...,va-overlap-0,va,OVERLAP
2,-78.47170579463713;38.07024061633331;va-albema...,va-overlap-0,va,OVERLAP
3,-78.4742397141423;38.06915127839967;va-albemar...,va-overlap-0,va,OVERLAP
4,-78.47615176924911;38.066440425054644;va-albem...,va-overlap-0,va,OVERLAP


In [8]:
newdf.to_csv("VA_Overlap_Parsed.csv", index=False)